In [1]:
 # Installing
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.12.0 --quiet
!pip install transformers --quiet
!pip install -q sentencepiece

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
ERROR: Could not build wheels for gensim, which is required to install pyproject.toml-based projects


In [2]:
#@title Installs

!pip install datasets --quiet

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_hub as hub
#import tensorflow_datasets as tfds
#import tensorflow_text as tf_text


from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration, TFAutoModelForSequenceClassification

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt
import transformers

import re

#This continues to work with gensim 3.8.3.  It doesn't yet work with 4.x.
#Make sure your pip install command specifies gensim==3.8.3
import gensim

In [4]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Running on TPU  ['10.3.162.202:8470']


In [5]:
#@title Loading dataset

# Load the dataset
from datasets import load_dataset
import pandas as pd
npc_train = load_dataset("amaydle/npc-dialogue", split="train")
npc_test = load_dataset("amaydle/npc-dialogue", split="test")

# Automatically splits it into train and test for you - let's ignore that for now and just combine them as one

# First, transform them into pandas DFs
train = pd.DataFrame(data = {'name': npc_train['Name'], 'bio':npc_train['Biography'], 'query':npc_train['Query'], 'response':npc_train['Response'], 'emotion':npc_train['Emotion']})
test = pd.DataFrame(data = {'name': npc_test['Name'], 'bio':npc_test['Biography'], 'query':npc_test['Query'], 'response':npc_test['Response'], 'emotion':npc_test['Emotion']})



# Just use the training dataset for now
# npc = pd.concat([train, test])

In [6]:
train['query'] = 'question: ' + train['query']
train['response'] = 'response: ' + train['response']
train['dialogue'] = train['query'] + ' ' + train['response']
train = train.groupby(['name', 'bio'])['dialogue'].apply(lambda x: ' '.join(x)).reset_index()
train['last_response_index'] = train['dialogue'].apply(lambda x: x.rfind('response: '))

def subset_str(my_string, my_index):
  return my_string[my_index:]

def remove_last_response(my_string, my_index):
  return my_string[:my_index]

train['last_response'] = train[['dialogue', 'last_response_index']].apply(lambda x: subset_str(*x), axis=1)
train['dialogue'] = train[['dialogue', 'last_response_index']].apply(lambda x: remove_last_response(*x), axis=1)

In [7]:
test['query'] = 'question: ' + test['query']
test['response'] = 'response: ' + test['response']
test['dialogue'] = test['query'] + ' ' + test['response']
test = test.groupby(['name', 'bio'])['dialogue'].apply(lambda x: ' '.join(x)).reset_index()
test['last_response_index'] = test['dialogue'].apply(lambda x: x.rfind('response: '))

test['last_response'] = test[['dialogue', 'last_response_index']].apply(lambda x: subset_str(*x), axis=1)
test['dialogue'] = test[['dialogue', 'last_response_index']].apply(lambda x: remove_last_response(*x), axis=1)
test

,name,bio,dialogue,last_response_index,last_response
0,Adalyn the Huntress,Adalyn is a fearless and skilled hunter from t...,question: Do you have any hunting techniques t...,246,"response: No, I do not regret any of my hunts...."
1,Agent 47,"Agent 47 is a genetically engineered assassin,...",question: How did Agent 47 become an assassin?...,152,response: Not that I'm aware of.
2,Aiden Stone,Aiden is a skilled thief who grew up on the st...,"question: ""How do you feel about people who un...",206,"response: ""The most important lesson I've lear..."
3,Alistair the Enchanter,Alistair is a powerful wizard who lived in the...,question: Have you ever been betrayed by someo...,63,"response: Yes, but learned to forgive and move..."
4,Althea,Althea is a half-elf ranger with a fierce inde...,"question: Althea, have you ever faced a danger...",1004,response: (Pauses) There have been a few times...
...,...,...,...,...,...
79,Vincenzo,"Vincenzo is a cunning and charismatic thief, w...",question: What is your greatest accomplishment...,137,"response: ""Honesty, for fools, not me."""
80,Wednesday Addams,Wednesday Addams is a teenage goth and the dau...,question: What is your opinion on love? respon...,344,"response: ""Justice, important, must be served."""
81,Zara the Brave,"Zara was born in a small village, but her curi...",question: What motivates Zara to keep fighting...,559,response: Zara believes that the kingdom is in...
82,Zephyr,Zephyr is a mischievous fairy who loves playin...,question: What motivates you to play pranks on...,524,"response: Yes, I have. It was a difficult expe..."


Data Preparation

In [8]:
# Pre-processing steps of the input claim for T5 classification task

def prep_for_t5(input):
  # pre-pend the data with the task: since we're not using one of T5's built in tasks we can just make up our own (this step is optional)
  input = 'answer: ' + input
  return input

# Apply to the dataset - using the combined text that uses other metadata as well (such as the bio)
def concatenate_input(df):
  df['input'] = df['name'] + ' ' + df['bio'] + ' ' + df['dialogue']
  return df

train = concatenate_input(train)
train['input'] = train['input'].apply(lambda x: prep_for_t5(x))
test = concatenate_input(test)
test['input'] = test['input'].apply(lambda x: prep_for_t5(x))

train

,name,bio,dialogue,last_response_index,last_response,input
0,Adalyn the Huntress,Adalyn is a fearless and skilled hunter from t...,question: Have you ever hunted in a different ...,2589,"response: I hunt all sorts of animals, from de...",answer: Adalyn the Huntress Adalyn is a fearle...
1,Agent 47,"Agent 47 is a genetically engineered assassin,...",question: Does Agent 47 have any allies? respo...,1593,response: Not that I know of.,answer: Agent 47 Agent 47 is a genetically eng...
2,Aiden Stone,Aiden is a skilled thief who grew up on the st...,question: Can you tell me about the first time...,3552,"response: ""I enjoy spending time in nature, an...",answer: Aiden Stone Aiden is a skilled thief w...
3,Alistair the Enchanter,Alistair is a powerful wizard who lived in the...,question: Can you tell me about a magical item...,1727,"response: Encountered many, but dragons are my...",answer: Alistair the Enchanter Alistair is a p...
4,Althea,Althea is a half-elf ranger with a fierce inde...,"question: Althea, have you ever been betrayed ...",3470,response: (Shudders) There was this one time I...,answer: Althea Althea is a half-elf ranger wit...
...,...,...,...,...,...,...
97,Vincenzo,"Vincenzo is a cunning and charismatic thief, w...",question: What is your opinion on justice? res...,1506,"response: ""No, steal from rich, not poor.""",answer: Vincenzo Vincenzo is a cunning and cha...
98,Wednesday Addams,Wednesday Addams is a teenage goth and the dau...,question: Have you ever felt scared? response:...,1390,"response: ""Norms, boring, ignore them.""",answer: Wednesday Addams Wednesday Addams is a...
99,Zara the Brave,"Zara was born in a small village, but her curi...",question: How does Zara feel about working wit...,3996,"response: Zara doesn't have much free time, bu...",answer: Zara the Brave Zara was born in a smal...
100,Zephyr,Zephyr is a mischievous fairy who loves playin...,question: Do you believe in true love? respons...,3348,"response: I don't really fear anything, to be ...",answer: Zephyr Zephyr is a mischievous fairy w...


In [9]:
train = train[['input', 'last_response']].rename(columns={'last_response':'response'})
train = train.to_dict('records')

test = test[['input', 'last_response']].rename(columns={'last_response':'response'})
test = test.to_dict('records')

In [10]:
train[0]

{'input': "answer: Adalyn the Huntress Adalyn is a fearless and skilled hunter from the forest. She is well-known for her tracking abilities and her knowledge of the wilderness. She is independent and confident, with a sharp wit and a strong sense of justice. question: Have you ever hunted in a different environment? response: Yes, I have hunted in many different environments, from forests to mountains to deserts. question: Do you hunt for sport or food? response: Both, but mostly for food. question: What do you like most about hunting? response: The thrill of the chase. question: Have you ever encountered a magical creature during your hunts? response: Yes, I've hunted enchanted deer and unicorns. question: Have you ever been injured while hunting? response: Yes, I have been injured many times. Hunting can be dangerous, but I always heal quickly. question: Why do you hunt? response: Hunting provides food and resources for my village, and it keeps me sharp and skilled. question: Do you

In [11]:
# Encode the dataset
def encode(example,
           encoder_max_len, decoder_max_len):

    input = example['input']
    label = example['response']

    encoder_inputs = t5_tokenizer(input, return_tensors='tf', max_length=encoder_max_len, padding='max_length', truncation=True)

    decoder_inputs = t5_tokenizer(label, return_tensors='tf', max_length=decoder_max_len, padding='max_length', truncation=True)

    input_ids = encoder_inputs['input_ids'][0]
    input_attention = encoder_inputs['attention_mask'][0]
    target_ids = decoder_inputs['input_ids'][0]
    target_attention = decoder_inputs['attention_mask'][0]

    outputs = {'input_ids':input_ids, 'attention_mask': input_attention,
               'labels':target_ids, 'decoder_attention_mask':target_attention}
    return outputs

In [12]:
# organize the inputs as a dictionary to convert to a tf dataset
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
ds = [encode(x, encoder_max_len = 512, decoder_max_len = 50) for x in train]
test_ds = [encode(x, encoder_max_len = 512, decoder_max_len = 50) for x in test]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# Convert to tf dataset

# Code borrowed from https://colab.research.google.com/github/snapthat/TF-T5-text-to-text/blob/master/snapthatT5/notebooks/TF-T5-Datasets%20Training.ipynb#scrollTo=TJVLbHbelMki
# and drew inspiration from https://stackoverflow.com/questions/68567630/converting-a-list-of-dictionaries-to-a-tf-dataset
def to_tf_dataset(dataset):
  input_ids = tf.data.Dataset.from_tensor_slices([d['input_ids'] for d in dataset])
  input_attention = tf.data.Dataset.from_tensor_slices([d['attention_mask'] for d in dataset])
  labels = tf.data.Dataset.from_tensor_slices([d['labels'] for d in dataset])
  decoder_attention_mask = tf.data.Dataset.from_tensor_slices([d['decoder_attention_mask'] for d in dataset])
  ds = tf.data.Dataset.zip((input_ids, input_attention, labels, decoder_attention_mask))
  ds = ds.map(lambda x, y, z, l: {"input_ids": x, "attention_mask": y,
                                "labels": z, "decoder_attention_mask": l})
  return ds

In [14]:
# 4 datasets - train/test for six way and binary classification tasks
ds = ds[0:1722]
train_tf = to_tf_dataset(ds)
test_tf = to_tf_dataset(test_ds)

In [15]:
# compile the models
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
#import T5 model (using small to minimize run time)

    # Batch the datasets
  def batch_dataset(dataset, batch_size=7):
    dataset = dataset.batch(batch_size)
    return dataset

  train_tf = batch_dataset(train_tf)
  test_tf = batch_dataset(test_tf)

  model = TFT5ForConditionalGeneration.from_pretrained('t5-small') # using small to prevent GPU from crashing
  learning_rate = 0.001
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [16]:
test[20]['input']

'answer: Casey Bowman Casey Bowman is a highly skilled ninja who grew up in a traditional ninja family. question: How does Casey Bowman view the outside world? response: No mention. question: How does Casey Bowman maintain his ninja skills? response: Hones skills to perfection. question: How does Casey Bowman feel about his ninja duties? '

In [17]:
encoded_query = t5_tokenizer(test[20]['input'],
                         return_tensors='tf', padding='max_length', truncation=True, max_length=512)
input_ids = encoded_query["input_ids"]
attention_mask = encoded_query["attention_mask"]
generated_answer = model.generate(input_ids)
t5_tokenizer.decode(generated_answer.numpy()[0])



/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<pad> True</s>'

In [18]:
test[13]['input']

'answer: Arinon the Bold Arinon is a valiant knight, known for his bravery and unwavering sense of justice. He was born in a small village, but always dreamed of being a knight. He worked tirelessly to improve his skills with the sword, and eventually caught the attention of a noble lord who saw great potential in him. Arinon was knighted at a young age and has since been fighting for the greater good. question: Have you ever tasted the local ale at The Rusty Tankard Inn? '

In [19]:
encoded_query = t5_tokenizer(test[13]['input'],
                         return_tensors='tf', padding='max_length', truncation=True, max_length=512)
input_ids = encoded_query["input_ids"]
attention_mask = encoded_query["attention_mask"]
generated_answer = model.generate(input_ids)
t5_tokenizer.decode(generated_answer.numpy()[0])

'<pad> True</s>'

In [20]:
test[13]['response']

"response: I'm afraid I can't say I have. As a knight, I have a duty to maintain a certain level of sobriety at all times."

In [21]:
sentence_encoder_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "UNCOMPRESSED"
sentence_encoder = hub.load(sentence_encoder_url)
def embed(input):
  return sentence_encoder(input)

In [22]:
def encode_response(response):
  response_ids = t5_tokenizer(response,
                         return_tensors='tf', padding='max_length', truncation=True, max_length=512)
  response_ids = response_ids["input_ids"]
  return response_ids

In [23]:
test_inputs = [x['input'] for x in test]
test_labels = [x['response'] for x in test]

In [24]:
# Source: https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

sentence_embedding_labels = [embed([x]) for x in test_labels]
encoded_inputs = [encode_response(x) for x in test_inputs]
encoded_predictions = [model.generate(x) for x in encoded_inputs]


In [25]:
# Source: https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder

decoded_predictions = [t5_tokenizer.decode(x.numpy()[0]) for x in encoded_predictions]
decoded_predictions = [x.replace("<pad>", "").replace("</s>", "") for x in decoded_predictions]
sentence_embedding_predictions = [encode_response([x]) for x in decoded_predictions]


In [26]:
def calc_similarity(encoded_sentence_1, encoded_sentence_2):
  similarity = np.inner(encoded_sentence_1, encoded_sentence_2)
  return similarity

cosine

NameError: ignored

In [27]:
!pip install -q torchmetrics
from torchmetrics.text.rouge import ROUGEScore
preds = decoded_predictions
target = test_labels
rouge = ROUGEScore()
from pprint import pprint
pprint(rouge(preds, target))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 5.9 MB/s eta 0:00:00
{'rouge1_fmeasure': tensor(0.0009),
 'rouge1_precision': tensor(0.0119),
 'rouge1_recall': tensor(0.0005),
 'rouge2_fmeasure': tensor(0.),
 'rouge2_precision': tensor(0.),
 'rouge2_recall': tensor(0.),
 'rougeL_fmeasure': tensor(0.0009),
 'rougeL_precision': tensor(0.0119),
 'rougeL_recall': tensor(0.0005),
 'rougeLsum_fmeasure': tensor(0.0009),
 'rougeLsum_precision': tensor(0.0119),
 'rougeLsum_recall': tensor(0.0005)}
